In [1]:
#!jt -t monokai -T -N -kl
#!jt -r

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

In [ ]:
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 12)
#plt.style.use('dark_background')
plt.style.use('ggplot')

In [ ]:
url_country = 'https://github.com/datasets/covid-19/raw/master/data/countries-aggregated.csv'
country_df = pd.read_csv(url_country)[['Date', 'Country', 'Deaths']]

In [ ]:
W = pd.DataFrame(country_df.groupby('Date').sum())
W['Country'] = ['World' for _ in range(len(W))]
W['Date'] = W.index
country_df = country_df.append(W)

In [ ]:
Deaths = pd.DataFrame()
Deaths['Date'] = country_df.Date.unique()
Deaths['Date'] = pd.to_datetime(Deaths.Date)
#Deaths = Deaths.set_index('Date')

In [ ]:
countries = country_df['Country'].unique()
for c in countries:
    cun = country_df[country_df.Country == c].Deaths
    Deaths[c] = list(cun)
Deaths = Deaths.set_index('Date')

In [ ]:
Important_countries = ['India', 'World', #'World without China',
                      'US', 'Germany', 'Italy', 'Spain', 'China',
                      'United Kingdom', 'France',
                      'Korea, South', 'Iran']#[::-1]
colors = {'India': 'Blue', 'World': 'Blue', 'World without US': 'Green', 'US': 'Black', 'China': 'red', 'Spain': 'yellow',
         'Italy': 'grey', 'Iran': 'brown', 'Ireland': 'violet', 'United Kingdom': 'Purple', 'Korea, South': 'Black',
         'France': 'orange', 'Germany': 'Green'}
def plotter(DF, name, log = True):
    plt.figure(figsize = (14, 7))
    if log:
        plt.yscale('symlog')
    if len(DF.columns)>len(Important_countries):
        columns = Important_countries
    else:
        columns = DF.columns
    for col in columns:
        plt.plot(DF.index,
                 DF[col],
                 label = col,
                 linestyle = '-.' if 'World' in col else '-',
                 color = colors[col] if col in colors else 'pink', 
                 #alpha = 0.6
                )
    plt.title(name+' wrt Date in Countries')
    plt.xlabel('Date')
    plt.ylabel(name)
    plt.legend()
    plt.show()

In [ ]:
def plot2(DF, DF2, name1, name2 = 'Deaths', log = True):
    plt.figure(figsize = (14, 7))
    if log:
        plt.xscale('log')
        if log == 'symlog':
            plt.yscale('symlog')
        else:
            plt.yscale('log')
    for col in Important_countries:
        plt.plot(DF[col],
                 DF2[col],
                 label = col,
                 linestyle = '-.' if 'World' in col else '-',
                 color = colors[col] if col in colors else 'pink', 
                 #alpha = 0.6
                )
    plt.title(f'{name1} wrt {name2} in Countries')
    plt.xlabel(name2)
    plt.ylabel(name1)
    plt.legend()
    plt.show()

In [ ]:
Deaths.tail(14)[Important_countries]

In [ ]:
plotter(Deaths[Important_countries].tail(50), 'Deaths')

In [ ]:
plotter(Deaths[['India']], 'Deaths', False)
#Deaths[['India']].tail(50)

In [ ]:
plotter(Deaths[['Korea, South']], 'Deaths', False)

In [ ]:
plotter(Deaths[['Germany']], 'Deaths', False)

In [ ]:
Removed = np.round(Deaths/0.02).astype('int')
Removed[Important_countries].tail(14)

In [ ]:
Biweekly_Deaths = Deaths - Deaths.shift(14)
Biweekly_Deaths = Biweekly_Deaths.fillna(0).astype('int')
Biweekly_Deaths[Important_countries].tail(14)

In [ ]:
plot2(Deaths, Biweekly_Deaths, 'Biweekly Deaths')

In [ ]:
plotter(Biweekly_Deaths[Important_countries].tail(50), 'Biweekly Deaths')

In [ ]:
# daily increase in biweekly deaths
Increase_Biweekly = Biweekly_Deaths-Biweekly_Deaths.shift(14)
Increase_Biweekly = Increase_Biweekly.fillna(0).astype('int')
Increase_Biweekly[Important_countries].tail(14)

In [ ]:
plotter(Increase_Biweekly, 'Increase Biweekly', False)

In [ ]:
plot2(Deaths, Increase_Biweekly, 'Increase Biweekly Deaths', log = 'symlog')

In [ ]:
Slope = np.round(Increase_Biweekly/Biweekly_Deaths, 4)
Slope[Important_countries].tail(14)

In [ ]:
plotter(Slope[[c for c in Important_countries if c!= 'China']].tail(50), 'Slope', True)

In [ ]:
Biweekly_Growth = np.round(Biweekly_Deaths/Biweekly_Deaths.shift(14), 4)
Biweekly_Growth[Important_countries].tail(14)

In [ ]:
plotter(Biweekly_Growth[Important_countries].tail(14), 'Biweekly_Growth', True)

In [ ]:
Confirmed = np.round(Removed * Biweekly_Growth)#.astype('int')
#Confirmed = Confirmed.tail(14).astype('int')
Confirmed[Important_countries].tail(14)

In [ ]:
Daily_Deaths = Deaths-Deaths.shift(1)
Daily_Deaths = Daily_Deaths.fillna(0).astype('int')
Daily_Deaths[Important_countries].tail(14)

In [ ]:
plot2(Deaths, Daily_Deaths, 'Daily Deaths')